In [11]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
nltk.download('punkt')

y_book_array = ["HP1", "HP2", "HP3", "HP4"]


# Function to read and preprocess text
def read_and_preprocess(files):
    corpus = []
    book_tokens = []
   
    for file in files:
        with open(file, 'r') as f:
            text = f.read().lower()
            # Split text into pages
            pages = text.split('\n')
            
            book_pages_tokens = []
            for page in pages:
                # Remove punctuation
                page = re.sub(r'[^\w\s]', '', page)
                # Tokenize
                tokens = word_tokenize(page)
                corpus.extend(tokens)
                book_pages_tokens.append(tokens)
            
            book_tokens.append(book_pages_tokens)
    return corpus, book_tokens

# Specify the files for the seven books
files = ['HarryPotter/HP1.txt', 'HarryPotter/HP2.txt', 'HarryPotter/HP3.txt', 'HarryPotter/HP4.txt']

# Preprocess the text
# tokens is now a 3d array with each sub array has sub-subarrays that are each page for each book
corpus, tokens = read_and_preprocess(files)

[nltk_data] Downloading package punkt to /home/zachs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def extract_ngrams_from_tokens(tokens, book,  n):
    # Step 1: Initialize an empty list to store N-grams
    n_grams = []
    y_array = []
    # Step 2: Generate N-grams
    for i in range(len(tokens) - n + 1):
        n_gram = ' '.join(tokens[i:i+n])
        n_grams.append(n_gram)
        y_array.append(book)
    return n_grams, y_array

for i in range(len(files)):
    if i ==0:
        pages_trigramed = []
        pages_y_array = []
        # for each page 
        for j in range(len(tokens[i])): 
            trigrams, trigrams_y_array = extract_ngrams_from_tokens(tokens[i][j],y_book_array[i], 3)
            pages_trigramed.append(trigrams)
            pages_y_array.append(y_book_array[i])
        x_y_trigrams = pd.DataFrame({'trigrams': pages_trigramed, 'book': pages_y_array})
    else:
        pages_trigramed = []
        pages_y_array = []
        for j in range(len(tokens[i])): 
            trigrams, trigrams_y_array = extract_ngrams_from_tokens(tokens[i][j],y_book_array[i], 3)
            pages_trigramed.append(trigrams)
            pages_y_array.append(y_book_array[i])
        book_x_y_trigrams = pd.DataFrame({"trigrams": pages_trigramed, "book": pages_y_array})
        x_y_trigrams = pd.concat([x_y_trigrams, book_x_y_trigrams], ignore_index=True)
    
display(x_y_trigrams)

,trigrams,book
0,"[mr and mrs, and mrs dursley, mrs dursley of, ...",HP1
1,"[met for several, for several years, several y...",HP1
2,"[the cat it, cat it stared, it stared back, st...",HP1
3,"[calls and shouted, and shouted a, shouted a b...",HP1
4,"[he found it, found it a, it a lot, a lot hard...",HP1
...,...,...
2018,"[harry potter and, potter and the, and the gob...",HP4
2019,"[harry potter and, potter and the, and the gob...",HP4
2020,"[harry potter and, potter and the, and the gob...",HP4
2021,"[harry thanks george, thanks george muttered, ...",HP4


In [13]:

X_train_val, X_test, y_train_val, y_test = train_test_split(x_y_trigrams["trigrams"], x_y_trigrams["book"], test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)
